In [14]:
import numpy as np
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account
import pyarrow
from pandas import Timestamp
import itertools
from datetime import datetime
import datetime
import time
from tqdm import tqdm
from datetime import datetime
from joblib import Parallel, delayed
import collections
import random
import pyarrow.parquet as pq

In [2]:
# my credential file
#key_path = "/Users/yuxuanyang/Downloads/encoded-adviser-131008-f2e09e85c3b3.json"
key_path = "/Users/yuxuanyang/Downloads/encoded-adviser-131008-7129911936a3.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [ ]:
query= """
    select *
    from `encoded-adviser-131008.yang_20230130.20000users`
"""
query_job = client.query(query)  # Make an API request.
users = query_job.to_dataframe()
pd.to_pickle(users, '/Users/yuxuanyang/Downloads/DSIN-master/fpro_data/sample_users.pkl')

In [ ]:
query= """
    select *
    from `encoded-adviser-131008.yang_20230130.apps`
"""
query_job = client.query(query)  # Make an API request.
apps = query_job.to_dataframe()
pd.to_pickle(apps, '/Users/yuxuanyang/Downloads/DSIN-master/fpro_data/apps.pkl')

In [ ]:
query= """
    select a.user_id as user_id, a.datetime as datetime, a.app_id as app_id, b.cate as cate, b.store as store
    from `encoded-adviser-131008.yang_20230130.logs_8days_20000users` a
    left join `encoded-adviser-131008.yang_20230130.apps` b
    on a.app_id = b.app_id
    order by cate
"""
query_job = client.query(query)  # Make an API request.
user_logs = query_job.to_dataframe()
#pd.to_pickle(user_logs, '/Users/yuxuanyang/Downloads/DSIN-master/fpro_data/sample_user_logs.pkl')

In [65]:
query= """
    select a.user_id as user_id, a.app_id as app_id, count(a.datetime) as access_count
    from `encoded-adviser-131008.yang_20230130.foreground_log_201906` a
    inner join `encoded-adviser-131008.yang_20230130.20000users` b
    on a.user_id = b.user_id
    group by user_id, app_id
    order by access_count
"""
query_job = client.query(query)  # Make an API request.
user_apps = query_job.to_dataframe()
#pd.to_pickle(user_logs, '/Users/yuxuanyang/Downloads/DSIN-master/fpro_data/sample_user_logs.pkl')

In [ ]:
def unix_to_timestamp(ut):
    return datetime.fromtimestamp(ut).strftime('%Y-%m-%d %H:%M:%S')
def timestamp_to_unix(ts):
    return time.mktime(ts.timetuple())

In [17]:
user_logs = pd.read_parquet('/Volumes/PortYoox/Deep_learning_models/DSIN-master/fpro_data/sample_user_logs.parquet')

In [4]:
user_logs = user_logs.reset_index().drop(['index','cate', 'store'],axis = 1)
user_logs 

,user_id,app_id,time_stamp
0,6d6f064d61b71f3db3154503df85da94dff735d0,com.nttdocomo.android.schedulememo,1.559401e+09
1,639a86d0135110a60e4e31df34f97734c810bfdc,com.twitter.android,1.559401e+09
2,bc841c64c66749ff715b4c8ae37d6ff498b27f67,jp.naver.line.android,1.559401e+09
3,98d7f110898790fcfbbd9d752a54eeab4a421082,com.instagram.android,1.559401e+09
4,8aa80b417af59f41b34180df31e17bae3fb0185c,com.twitter.android,1.559401e+09
...,...,...,...
74143487,1556238018b5a847c7db80290c8b996923fe657c,jp.naver.line.android,1.560092e+09
74143488,c7e0fe710e5c36b5d35063c3588002d6d6b63933,jp.eure.android.pairs,1.560092e+09
74143489,45ddb3cada140fb8dba21195d9abc19218b4adc4,jp.co.yahoo.gyao.android.app,1.560092e+09
74143490,038695ebcc685e85499eac7b2a06a1896db4f1ee,com.apps.power.super.clean.security.master,1.560092e+09


In [20]:
def gen_negative_samples(user, log, sample_by_popular):
    log = log.reset_index()
    # for each access log, the negative samples were generated by the other apps in the user's device but did not click
    app_counter = collections.Counter(log['app_id'].tolist())
    app_id_list = []
    time_list = []
    for i in range(len(log)):
        curr_app = log.iloc[i]['app_id']
        time_stamp = log.iloc[i]['time_stamp']
        if sample_by_popular == False:
            other_apps = [a for a in list(app_counter.keys()) if a != curr_app]
            if other_apps:
                sample_negative_app = random.choice(other_apps)
                app_id_list.append(sample_negative_app)
                time_list.append(time_stamp)
              #  negative_df.loc[len(negative_df)]= [user, sample_negative_app, time_stamp, 0]
        elif sample_by_popular == True:
            other_apps = [a for a in list(app_counter.keys()) if a != curr_app]
            popular_sorted_apps = sorted(other_apps, key = lambda x: app_counter[x])
            if popular_sorted_apps:
                sample_negative_app = popular_sorted_apps[-1]
                app_id_list.append(sample_negative_app)
                time_list.append(time_stamp)
            #   negative_df.loc[len(negative_df)]= [user, sample_negative_app, time_stamp, 0]
    l = len(app_id_list)
    negative_df = pd.DataFrame({'user_id': [user]*l,
                                'app_id': app_id_list,
                                'time_stamp': time_list,
                                'clk': [0]*l})
    return negative_df

def applyParallel(dfGrouped, func, sample_by_popular):
    res = Parallel(n_jobs= -1)(delayed(func)(name, group, sample_by_popular) \
                                 for name, group in tqdm(dfGrouped))
    return pd.concat(res)


In [27]:
negative_samples_by_random = applyParallel(user_logs.groupby('user_id'), gen_negative_samples, False)

100%|██████████| 20000/20000 [18:02<00:00, 18.48it/s]


In [34]:
negative_samples_by_random = negative_samples_by_random.reset_index().drop(['index'],axis = 1)

In [35]:
pd.to_pickle(negative_samples_by_random, '../fpro_data/negative_samples_by_random.pkl')

In [29]:
negative_samples_by_popular = applyParallel(user_logs.groupby('user_id'), gen_negative_samples, True)

100%|██████████| 20000/20000 [19:19<00:00, 17.26it/s]


In [36]:
negative_samples_by_popular = negative_samples_by_popular.reset_index().drop(['index'],axis = 1)
negative_samples_by_popular 

,user_id,app_id,time_stamp,clk
0,00066192c8801d12eb8b412128df3b0a75ca0fee,com.twitter.android,1.559401e+09,0.0
1,00066192c8801d12eb8b412128df3b0a75ca0fee,com.twitter.android,1.559401e+09,0.0
2,00066192c8801d12eb8b412128df3b0a75ca0fee,com.twitter.android,1.559401e+09,0.0
3,00066192c8801d12eb8b412128df3b0a75ca0fee,com.twitter.android,1.559401e+09,0.0
4,00066192c8801d12eb8b412128df3b0a75ca0fee,com.twitter.android,1.559401e+09,0.0
...,...,...,...,...
74143033,fff2935469f41bc4c85c5fbb01aaf941daa7d0ad,jp.naver.line.android,1.560002e+09,0.0
74143034,fff2935469f41bc4c85c5fbb01aaf941daa7d0ad,jp.naver.line.android,1.560004e+09,0.0
74143035,fff2935469f41bc4c85c5fbb01aaf941daa7d0ad,com.sonymobile.home,1.560004e+09,0.0
74143036,fff2935469f41bc4c85c5fbb01aaf941daa7d0ad,com.sonymobile.home,1.560004e+09,0.0


In [37]:
pd.to_pickle(negative_samples_by_popular , '../fpro_data/negative_samples_by_popular.pkl')

In [19]:
user_logs.replace(to_replace=[None], value= ['nan'], inplace=True)

In [7]:
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from tqdm import tqdm
from sklearn.ensemble import IsolationForest
from pandas.testing import assert_frame_equal

FRAC = 0.25
user_logs = pd.read_pickle('../fpro_data/sample_user_logs.pkl')
user_logs.replace(to_replace=[None], value= ['nan'], inplace=True)
def gen_intervals(user, user_log, groupby_sdt):
    sorted_log = user_log.sort_values(by = ['time_stamp'])
    if len(sorted_log) > 1:
        intervals = {}
        time1 = sorted_log.time_stamp.to_numpy()[:-1]
        time2= sorted_log.time_stamp.to_numpy()[1:]
        name = sorted_log[groupby_sdt].to_numpy()[:-1]
        diffs = time2 - time1 
        return pd.DataFrame({groupby_sdt: name, 'interval': diffs})

def applyParallel(dfGrouped, func, groupby_sdt):
    res = Parallel(n_jobs= -1)(delayed(func)(name, group, groupby_sdt) for name, group in tqdm(dfGrouped))
#log.iloc[:100].groupby('user').apply(gen_intervals)
    return pd.concat(res)
#删除异常值 用异常分数的0.5作为阀值
def iforest_remove_outliner(name, intervals_groupby, groupby_sdt):
    sll = intervals_groupby.interval.tolist()

    X = np.array(sll).reshape(len(sll),1)
    clf = IsolationForest(random_state=0).fit(X)
    predict_result=clf.predict(X)
    tl=[]
    for i in range(len(sll)):
        if predict_result[i] == 1:
            tl.append(sll[i])
    if len(tl) > 0:
        thresholds = pd.DataFrame({groupby_sdt : [name],'{}_threshold'.format(groupby_sdt):[max(tl)]})
        return thresholds

if __name__ == "__main__":
    cate_intervals = applyParallel(user_logs.groupby('user_id'), gen_intervals, 'cate')
    cate_thresholds = applyParallel(cate_intervals.groupby('cate'), iforest_remove_outliner, 'cate')
    pd.to_pickle(cate_thresholds.set_index('cate'), '../fpro_data/cate_intervals.pkl')
    brand_intervals = applyParallel(user_logs.groupby('user_id'), gen_intervals, 'store')
    brand_thresholds = applyParallel(brand_intervals.groupby('store'), iforest_remove_outliner, 'store')
    pd.to_pickle(brand_thresholds.set_index('store'), '../fpro_data/store_intervals.pkl')

 63%|██████▎   | 32/51 [00:18<00:10,  1.74it/s]/Users/yuxuanyang/.miniconda3/envs/tensorflow/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
  6%|▋         | 696/11132 [00:14<03:22, 51.46it/s]/Users/yuxuanyang/.miniconda3/envs/tensorflow/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
100%|██████████| 11132/11132 [06:11<00:00, 29.95it/s]


In [8]:
brand_thresholds.set_index('store')

,store_threshold
store,
(Andrei & Aleksandr Krupiankou),7.0
"(C)OUTWARD Co.,LTD.",6.0
(C)さふいずむ,63.0
"(THE)ONE of THEM, Inc.",129.0
(つ＿－*)｡οΟ,50.0
...,...
ＺＴＥジャパン株式会社,4.0
�� Magic Mirror ��,40.0
��100Pi Labs,35.0


In [9]:
cate_thresholds.set_index('cate')

,cate_threshold
cate,
ART_AND_DESIGN,75.0
AUTO_AND_VEHICLES,65.0
BEAUTY,28.0
BOOKS_AND_REFERENCE,122.0
BUSINESS,25.0
COMICS,93.0
COMMUNICATION,34.0
DATING,20.0
EDUCATION,50.0
